In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

'2.10.0'

# Softmax classification

* 임의의 Dataset 준비
* 3개의 클래스로 분류할 데이터 준비


In [6]:
x_data = [[1., 2., 1., 1.],
          [2., 1., 3., 2.],
          [3., 1., 3., 4.],
          [4., 1., 5., 5.],
          [1., 7., 5., 5.],
          [1., 2., 5., 6.],
          [1., 6., 6., 6.],
          [1., 7., 7., 7.]] # 8x4
y_data = [[0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 1., 0.],
          [0., 1., 0.],
          [0., 1., 0.],
          [1., 0., 0.],
          [1., 0., 0.]] # 8x3

x_test = [[1., 1., 1., 1.]]
y_test = [[0., 0., 1.]]


## 임의의 Data를 이용해서 3개의 클래스를 가지는 데이터셋 생성

In [7]:
#dataset을 선언합니다.
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
dataset = dataset.repeat(1).batch(8)

nb_classes = 3 # class의 개수입니다.

print(tf.Variable(x_data))
print(tf.Variable(y_data))

<tf.Variable 'Variable:0' shape=(8, 4) dtype=float32, numpy=
array([[1., 2., 1., 1.],
       [2., 1., 3., 2.],
       [3., 1., 3., 4.],
       [4., 1., 5., 5.],
       [1., 7., 5., 5.],
       [1., 2., 5., 6.],
       [1., 6., 6., 6.],
       [1., 7., 7., 7.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(8, 3) dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)>


In [8]:
#Weight and bias setting
W = tf.Variable(tf.random.normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name='bias') # 바이어스 3개 nb_classes
variables = [W, b]

tf.print(W,b)

[[-1.35441589 0.704549313 0.0366619118]
 [0.869187951 0.438422769 -0.534398437]
 [-0.0771029219 1.5658046 -0.101234496]
 [-0.274497509 1.42046595 1.26094651]] [-1.35441589 0.704549313 0.0366619118]


# 가설 설정

* 가설에서 예측한 값들을 이용해 예측값들을 확률로 표현한다.

## $$ y_k = \frac{exp(x_k)}{\sum_{i=1}^{n}exp(x_i)}  $$

In [9]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis_softmax(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

tf.print(hypothesis_softmax(tf.Variable(x_data)))

[[0.00135811116 0.992638111 0.00600378495]
 [7.79175593e-07 0.999749959 0.000249291]
 [3.35181127e-09 0.999907 9.29304078e-05]
 ...
 [3.9953485e-10 0.999996543 3.46176444e-06]
 [4.32878455e-10 1 1.33459421e-08]
 [2.36497037e-11 1 8.1207141e-10]]


## 가설을 검증할 Cross Entropy 함수를 정의합니다

$$
\begin{align}
cost(h(x),y) & = −\sum_{n=1}^{n} Y log(h(x))
\end{align}
$$

In [10]:
def loss_fn(hypothesis, labels):
    loss = tf.keras.losses.categorical_crossentropy(labels, hypothesis)
    # tf.keras.losses.binary_crossentropy(labels, hypothesis) # 이진분류용 CE Loss
    return loss

optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.01)

### 학습 진행

In [11]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
      loss_value = loss_fn(hypothesis_softmax(features),labels)
      grads = tape.gradient(loss_value, [W,b])
      optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
      if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, tf.reduce_mean(loss_value.numpy()).numpy()))

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x8x1x3xi1>'


Iter: 0, Loss: 6.8665
Iter: 100, Loss: 0.5365
Iter: 200, Loss: 0.4459
Iter: 300, Loss: 0.4027
Iter: 400, Loss: 0.3722
Iter: 500, Loss: 0.3480
Iter: 600, Loss: 0.3275
Iter: 700, Loss: 0.3097
Iter: 800, Loss: 0.2939
Iter: 900, Loss: 0.2797
Iter: 1000, Loss: 0.2668
Iter: 1100, Loss: 0.2550
Iter: 1200, Loss: 0.2442
Iter: 1300, Loss: 0.2343
Iter: 1400, Loss: 0.2251
Iter: 1500, Loss: 0.2166
Iter: 1600, Loss: 0.2087
Iter: 1700, Loss: 0.2013
Iter: 1800, Loss: 0.1944
Iter: 1900, Loss: 0.1879
Iter: 2000, Loss: 0.1819
Iter: 2100, Loss: 0.1762
Iter: 2200, Loss: 0.1708
Iter: 2300, Loss: 0.1657
Iter: 2400, Loss: 0.1609
Iter: 2500, Loss: 0.1564
Iter: 2600, Loss: 0.1521
Iter: 2700, Loss: 0.1481
Iter: 2800, Loss: 0.1442
Iter: 2900, Loss: 0.1405
Iter: 3000, Loss: 0.1370
Iter: 3100, Loss: 0.1337
Iter: 3200, Loss: 0.1305
Iter: 3300, Loss: 0.1275
Iter: 3400, Loss: 0.1246
Iter: 3500, Loss: 0.1218
Iter: 3600, Loss: 0.1191
Iter: 3700, Loss: 0.1166
Iter: 3800, Loss: 0.1141
Iter: 3900, Loss: 0.1118
Iter: 4000, 

## Sample 데이터를 넣고 테스트해봅시다.

In [12]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis_softmax(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[5.8078137e-04 4.6068985e-02 9.5335025e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


## 데이터를 이용해서 예측

In [13]:
b = hypothesis_softmax(x_test)
print(b)
print(tf.argmax(b, 1)) # tf.argmax(b, 1) 함수는 축 1을 따라 최댓값의 인덱스를 포함하는 텐서를 반환합니다.
print(tf.argmax(y_test, 1)) # matches with y_data


tf.Tensor([[2.4467602e-07 4.7964056e-04 9.9952018e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)


In [14]:
def accuracy_fn(hypothesis, labels):
    hypothesis = tf.argmax(hypothesis)
    hypothesis = tf.cast(hypothesis, dtype=tf.float32)
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    labels = tf.argmax(labels)
    labels = tf.cast(labels, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy
  
test_acc = accuracy_fn(hypothesis_softmax(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 1.0000
